In [ ]:
import warnings
import math
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from datetime import date
from statistics import mean, median
from time import sleep
from datetime import datetime, timedelta
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm
import numpy as np
import scipy.stats as stats
from scipy.stats import gmean
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import emoji
import torch
from slovnet.model.emb import NavecEmbedding
import matplotlib.pyplot as plt

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from navec import Navec
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
start = pd.read_excel(f"{151563780}_new_wave_1.xlsx")
start = pd.DataFrame(columns=start.columns)
start

In [ ]:
for i in [91198168,100752412,
          113148910,119906733,
          138216345,139140790,
          139823386,143874380,
          146523913,150048947,
          150606036,151973186]:
    basic = pd.read_excel(f"df_total_{i}.xlsx")
    start = start.append(basic)

In [ ]:
for i in [9943236,22003393,
          8601593,75212617,
          75640333,86283646,
          93199503,100339707,
          100817270,103487056,
          109266137,110259188,
          125406299,125436671,
          148056502,150136196,
          151563780,154532809]:
    basic = pd.read_excel(f"{i}_new_wave_1.xlsx")
    start = start.append(basic)

In [ ]:
count = 0
start['new_merge'] = 0
for i in tqdm(start['Артикул'].unique()):
    for j in start[start['Артикул']==i]['Отзыв_x'].unique():
        for k in start[(start['Артикул']==i) & (start['Отзыв_x']==j)]['id комментатора_x'].unique():
            count+=1
            start.loc[(start['Артикул']==i) &
                      (start['Отзыв_x']==j) &
                      (start['id комментатора_x']==k),'new_merge'] = count

In [ ]:
start['id_for_merge'] = start['id комментатора_x'].astype(str)+start['new_merge'].astype(str)

In [ ]:
start = start.dropna()
start = start.drop_duplicates()
start.head()
clear = start.groupby(['id_for_merge'])['Отзыв_y'].count().reset_index()
clear = clear.rename(columns={'Отзыв_y':'count_reviews'})

In [ ]:
len(set(start['Артикул']))

# cleaning + preprocessing

In [ ]:
mon = {'января':1,
       'февраля':2,
       'марта':3,
       'апреля':4,
       'мая':5,
       'июня':6,
       'июля':7,
       'августа':8,
       'сентября':9,
       'октября':10,
       'ноября':11,
       'декабря':12}

In [ ]:
def preprocess_text(text):
    text = emoji.replace_emoji(text, replace='')# удаление эмодзи
    text = re.sub(r'\d+', '', text) # удаление цифр
    text = re.sub(r'\s+', ' ', text).strip() # удаление лишних пробелов
    tokens = word_tokenize(text) # Токенизация
    tokens = [token.lower() for token in tokens] # Приведение к нижнему регистру
    lemmatizer = WordNetLemmatizer() # Лемматизация
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    table = str.maketrans('', '', string.punctuation + '’') # Очистка от знаков пунктуации и других символов
    tokens = [token.translate(table) for token in tokens]
    tokens = [token for token in tokens if len(token) > 0] # Удаление пустых токенов
    return tokens


def numbers(text):
    return int(re.findall(r'\d+', text)[0])
def correct_date(element):
    element = element.split(' ')
    no_comma = [rep.replace(',', '') for rep in element] 
    if len(no_comma) == 3:
        no_comma = no_comma[:2] + ['2023'] + no_comma[2:]
        
    corr_month = [*map(mon.get, no_comma, no_comma)]
    date = datetime(int(corr_month[2]),corr_month[1],
                    int(corr_month[0]),
                    int(corr_month[3][:2]),
                    int(corr_month[3][3:]))
    return date
def on_site(text):
    if ((text.find('месяца') >= 0) or 
        (text.find('месяцев') >= 0) or
        (text.find('месяц') >=0 )):
        numb = int(re.findall(r'\d+', text)[0])
    else:
        numb = int(re.findall(r'\d+', text)[0])*12
    return numb

In [ ]:
def new_func1_df(new_func1):
    new_func1['id'] = new_func1['id_for_merge'].apply(numbers)
    new_func1['id_srt'] = new_func1['Артикул'].astype(str) + '_' + new_func1['id'].astype(str)
    count = 0
    new_func1['sorted'] = 0
    for i in new_func1['id_srt'].unique():
        count+=1
        new_func1.loc[new_func1['id_srt']==i,'sorted'] = count

    new_func1['main_id'] = new_func1['Артикул'].astype(str) + '_' + new_func1['sorted'].astype(str)

    new_func1 = new_func1.rename(columns={
        'Отзыв_x':'main_review',
        'Отзыв_y':'other_review',
        'Оценка отзыва':'main_rate',
        'Оценки по отзывам':'other_rate',
        'id комментатора_x':'client_name',
        'Количество отзывов комментатора':'count_reviews',
        'Количество фотографий':'count_photo',
        'Заголовок':'item',
        'Количество отзывов':'main_item_reviews_count',
        'Артикул':'headphones_id'
    })
    
    new_func1['count_reviews'] = new_func1['count_reviews'].apply(numbers)
    new_func1['main_date'] = new_func1['Дата отзыва'].apply(correct_date)
    new_func1['other_date'] = new_func1['Дата отзыва комментатора'].apply(correct_date)
    new_func1['other_review_len'] = new_func1['other_review'].str.len()
    new_func1['main_review_len'] = new_func1['main_review'].str.len()
    new_func1['month_on_site'] = new_func1['Количество лет на сайте'].apply(on_site)
    new_func1['main_review_list'] = new_func1['main_review'].apply(preprocess_text)
    new_func1['other_review_list'] = new_func1['other_review'].apply(preprocess_text)
    new_func1['item_list'] = new_func1['item'].apply(preprocess_text)
    new_func1 = new_func1.sort_values(['sorted','other_date'],ascending=[True,False])
    
    sort_new_func1 = new_func1.copy()
    sort_new_func1 = sort_new_func1.reset_index(drop=True)
    sort_new_func1 = sort_new_func1[['headphones_id',
        'main_item_reviews_count', 'sorted','main_id','client_name','month_on_site','count_reviews',
        
        'main_review','main_review_list','main_date','main_review_len','main_rate',
        
        'item','item_list',
        
        'other_review','other_review_list','other_date','other_review_len','other_rate','count_photo'
    ]]
    return sort_new_func1

In [ ]:
new_func1 = start.copy()
sort_new_func1 = new_func1_df(new_func1)

In [ ]:
clear = sort_new_func1.groupby(['main_id'])['other_review'].count().reset_index()
clear = clear.rename(columns={'other_review':'count_reviews'})
sort_new_func1 = sort_new_func1.drop(columns='count_reviews')
sort_new_func1 = sort_new_func1.merge(clear,on='main_id',how='left')
sort_new_func1

# признак близости отзывов внутри клиента

In [ ]:
new_func2 = sort_new_func1.copy()
new_func2['date_median'] = 0
new_func2['date_std'] = 0
new_func2['rates_neg_share'] = 0
cosine_navec = {}

for i in tqdm(new_func2['main_id'].unique()):
    text_embd = {}
    gpt_embd = {}
    min_diff_revies = []
    rates = new_func2.loc[new_func2['main_id']==i]['other_rate'].value_counts().reset_index().copy()
    new_func2.loc[new_func2['main_id']==i,'rates_neg_share'] = 1-rates[(rates['index']==5) |
                                (rates['index']==4)]['other_rate'].sum()/rates['other_rate'].sum()
    for j in new_func2.loc[new_func2['main_id']==i].index:
        text_embd_list = []
        for k in new_func2.loc[(new_func2.index==j)]['other_review_list'][j]:
            try: 
                text_embd_list.append(navec[k])
            except KeyError:
                continue
        if len(text_embd_list) == 0:
            pass
        else:
            mean_arr = np.mean(np.array(text_embd_list), axis=0)    
            text_embd.update({j: mean_arr})
            if (abs((text_embd[j]+3).sum())>=0.1e-10) == False:
                text_embd.pop(j)
            
        if (new_func2.loc[new_func2['main_id']==i].index.to_list().index(j) !=
           (len(new_func2.loc[new_func2['main_id']==i].index)-1)):
            min_diff_revies.append((new_func2.loc[(new_func2['main_id']==i) &
                                                 (new_func2.index==j)]['other_date'][j] - 
                                    new_func2.loc[(new_func2['main_id']==i) &
                                                 (new_func2.index==j+1)]['other_date'][j+1]).total_seconds()/60)
            new_func2.loc[new_func2['main_id']==i,'date_median'] = int(np.median(min_diff_revies))
            new_func2.loc[new_func2['main_id']==i,'date_std'] = int(np.std(min_diff_revies))
    
    embeddings_list = list(text_embd.values())    
    if len(embeddings_list) !=0:
        matrix = cosine_similarity(embeddings_list)
        values = matrix[np.triu_indices(matrix.shape[0], k=1)]
        cosine_navec.update({i: values})
    else:
        cosine_navec.update({i: [0]})

In [ ]:
val = [val for sublist in cosine_navec.values() for val in sublist]

plt.figure(figsize=[12,7])
plt.hist(val, bins=50)
plt.title('Распределение косинусных близостей отзывов')
plt.xlabel('Значение косинусной близости')
plt.ylabel('Количество повторений')
plt.show()

In [ ]:
dd = pd.DataFrame.from_dict(cosine_navec, orient='index')

val = [val for sublist in cosine_navec.values() for val in sublist]
q70 = np.quantile(val, 0.85)
print(q70)

for i in cosine_navec.keys():
    dd.loc[dd.index==i,'q70_ratio'] = np.mean(cosine_navec[i]>q70)
    dd.loc[dd.index==i,'std'] = np.std(cosine_navec[i])

dd = dd[['q70_ratio','std']]
dd = dd.fillna(0)

In [ ]:
np.mean(val),np.median(val),np.quantile(val,0.7)

In [ ]:
new_func3 = new_func2.merge(dd,left_on='main_id',right_index=True,how='inner')

# признак повторямости товара среди клиентов

In [ ]:
def items_repeats(id_item_df):
    repeat = id_item_df['item'].value_counts().reset_index()
    repeat.head()
    item_embeddings = {}
    sim = {}
    for item in repeat['index']:
        item_name_prp = preprocess_text(item)
        word_embd = []
        for word in item_name_prp:
            try: 
                word_embd.append(navec[word])
            except KeyError:
                continue
        if len(word_embd) == 0:
            pass
        else:
            item_embeddings[item] = np.mean(np.array(word_embd), axis=0)
            if (abs((item_embeddings[item]+3).sum())>=0.1e-10) == False:
                item_embeddings.pop(item)
        
    items = list(item_embeddings.keys())
    for i in items:
        if item_embeddings[i].shape!=(300,):
            items.remove(i)
    repeat_cl = repeat[repeat['index'].isin(items)]
    for _, row in repeat_cl.iterrows():
        similar = []
        purchased_item = row['index']
        pur_item = item_embeddings[purchased_item]
        other_items = [item_embeddings[item] for item in items]
        similarities = cosine_similarity(pur_item.reshape(1, -1),
                                         other_items)[0]
        for i, _ in enumerate(similarities): 
            if similarities[i] > 0.9:
                similar.append(i)
        sim[purchased_item] = repeat_cl[repeat_cl.index.isin(similar)]['item'].sum()
    sim_df = pd.DataFrame([sim]).T.reset_index()
    
    count_items = repeat.merge(sim_df,how='left',on='index')
    count_items = count_items.rename(columns={
        'item':'value_c',
        'index':'item',
        0:'value_c_navec'
    })
    return count_items

In [ ]:
item_count_df = pd.DataFrame(columns=['headphones_id','item','value_c','value_c_navec'])
for i in tqdm(set(new_func3['headphones_id'])):
    df_tr = items_repeats(new_func3[new_func3['headphones_id']==i])
    df_tr['headphones_id'] = i
    item_count_df = item_count_df.append(df_tr)

In [ ]:
item_count_df = item_count_df.drop_duplicates()
item_count_df = item_count_df.reset_index(drop=True)

In [ ]:
item_count_df_corr = item_count_df.copy()
item_count_df_corr = item_count_df_corr.fillna(item_count_df_corr['value_c'])
item_count_df_corr['corr_counts'] = np.NaN
item_count_df_corr['item_mark'] = np.NaN
for i in tqdm(set(new_func3['headphones_id'])):
    item_count_df_corr.loc[(item_count_df_corr['headphones_id']==i)
                     & (item_count_df_corr['value_c']>
    item_count_df_corr[item_count_df_corr['headphones_id']==i]['value_c_navec'].max()*0.1),'corr_counts'] = item_count_df_corr[item_count_df_corr['headphones_id']==i]['value_c_navec'].max()
    
    item_count_df_corr.loc[(item_count_df_corr['headphones_id']==i)
                     & (item_count_df_corr['value_c']>
    item_count_df_corr[item_count_df_corr['headphones_id']==i]['value_c_navec'].max()*0.1),'item_mark'] = 1

In [ ]:
chechh = item_count_df_corr[item_count_df_corr['item_mark']==1].copy()
chechh

In [ ]:
chechh['item_list'] = chechh['item'].apply(preprocess_text)
ch = chechh.copy()
ch['delete'] = 0
for i in chechh.index:
    if ('микронаушник' in ch.loc[ch.index==i]['item_list'][i] or
        'микронаушники' in ch.loc[ch.index==i]['item_list'][i] or
        'наушники' in ch.loc[ch.index==i]['item_list'][i] or
        'наушник' in ch.loc[ch.index==i]['item_list'][i] or 
        'стереонаушники' in ch.loc[ch.index==i]['item_list'][i]):
        ch.loc[ch.index==i,'delete'] = 1

ch.loc[ch['delete']==0,'corr_counts'] = ch['value_c']
correct = ch[ch['delete']==0][['headphones_id','item','corr_counts']]

item_count_df_corr = item_count_df_corr.merge(correct,how='left',on=['headphones_id','item'])
item_count_df_corr['corr_counts_y'].fillna(item_count_df_corr['corr_counts_x'],inplace=True)
item_count_df_corr['corr_counts_y'].fillna(item_count_df_corr['value_c'], inplace=True)

In [ ]:
values_repeat = []
maxes = []
for i in item_count_df_corr['headphones_id'].unique():
    max_value_item = item_count_df_corr[item_count_df_corr['headphones_id']==i]['corr_counts_y'].max()
    print(i,max_value_item)
    item_count_df_corr.loc[item_count_df_corr.index==5004,'corr_counts_y']=498
    item_count_df_corr.loc[item_count_df_corr.index==32878,'corr_counts_y']=344
    item_count_df_corr.loc[item_count_df_corr.index==44569,'corr_counts_y']=291 
        
    values_repeat.append(list(item_count_df_corr[(item_count_df_corr['headphones_id']==i) &
                             (item_count_df_corr['corr_counts_y']!=max_value_item)]['corr_counts_y']))
    maxes.append(list(item_count_df_corr[(item_count_df_corr['headphones_id']==i) &
                             (item_count_df_corr['corr_counts_y']==max_value_item)]['item']))

In [ ]:
flat_list = []
for sublist in values_repeat:
    flat_list.extend(sublist)

In [ ]:
to_join_item = item_count_df_corr[['headphones_id','item','corr_counts_y']]

In [ ]:
new_func4 = new_func3.merge(to_join_item,how='left',on=['item','headphones_id'])
new_func4

# признак схожести основных отзывов между собой

In [ ]:
def main_cos_sim(id_item_df):
    math = id_item_df[['main_id','main_review_list']]
    math = math.drop_duplicates('main_id').reset_index(drop=True)
    
    embeddings_dict = {}
    cosine_main = {}
    for i in math['main_id']:
        main_embd_list = []
        for j in list(math[math['main_id']==i]['main_review_list'])[0]:
            try:   
                main_embd_list.append(navec[j])
            except KeyError:
                continue
         
        if len(main_embd_list) == 0:
                pass
        else:
            main_mean_arr = np.mean(np.array(main_embd_list), axis=0)    
            embeddings_dict.update({i: main_mean_arr})
            if (abs((embeddings_dict[i]+3).sum())>=0.1e-10) == False:
                print(i,math[math['main_id']==i]['main_review_list'],embeddings_dict[i])
                embeddings_dict.pop(i)
                
    cos_sim_dict = {}
    cos_q70 = {}
    
    for key in embeddings_dict.keys():
        embedding = embeddings_dict[key]
        
        cos_sim_list = []
        for other_key in embeddings_dict.keys():
            if other_key != key:
                other_embedding = embeddings_dict[other_key]
                cos_sim = cosine_similarity([embedding, other_embedding])
                main_values = cos_sim[np.triu_indices(cos_sim.shape[0], k=1)]
                cos_sim_list.append(main_values)
        
        mean_cos_sim = np.median(cos_sim_list)
        
        cos_sim_dict[key] = mean_cos_sim
        cos_q70[key] = round(np.mean(cos_sim_list>q70),2)
    return cos_q70

In [ ]:
new_func5 = pd.DataFrame(columns=new_func4.columns)
new_func5['main_simillar'] = 0
for i in tqdm(set(new_func4['headphones_id'])):
    new = new_func4.loc[new_func4['headphones_id']==i].copy()
    new['main_simillar'] = new['main_id'].map(main_cos_sim(new))
    new_func5 = new_func5.append(new)

In [ ]:
new_func5 = new_func5.sort_values(by=['sorted','other_date'],ascending=[True,False])

# признаки с фотографиями и длиной отзывов остальных

In [ ]:
cols_to_int = list(new_func5.dtypes[new_func5.dtypes==object].index)
cols_to_int = [col for col in cols_to_int if col not in ['main_review','other_review','main_id',
                                                         'item','main_review_list',
                                                         'other_review_list','item_list','client_name']]
new_func6 = new_func5.copy()
for i in cols_to_int:
    new_func6[i] = new_func6[i].astype(float)

In [ ]:
new_func6 = new_func6.drop(columns=['main_review_list','item_list','other_review_list'])
new_func6 = new_func6.drop_duplicates()
new_func6 = new_func6.dropna()

In [ ]:
gpt_df = new_func6.copy()
grouped = gpt_df.groupby(by=['headphones_id','main_id','sorted','month_on_site',
                'count_reviews',
                'main_review',
                'main_review_len',
                'main_rate',
                'date_median',
                'date_std',
                'rates_neg_share',
                'main_simillar',
                'q70_ratio','std'
                            ])
med = grouped['other_review_len','count_photo'].median()
med = med.rename(columns={'other_review_len':'med_other_review_len',
                     'count_photo':'med_count_photo'})

var = grouped['other_review_len','other_rate'].var()
var = var.rename(columns={'other_review_len':'var_other_review_len',
                     'other_rate':'var_other_rate'})

result = pd.concat([med, var], axis=1)
result = result.reset_index()
result = result.iloc[result['main_id'].drop_duplicates(keep='first').index]
result = result.reset_index(drop=True)
result = result.sort_values('sorted')

In [ ]:
result.head()

In [ ]:
value_c_navec = new_func4[['main_id','corr_counts_y']]
corr = pd.DataFrame(columns=value_c_navec.columns)
for i in tqdm(value_c_navec['main_id'].unique()):
    promo_df = value_c_navec[value_c_navec['main_id']==i]
    corr = corr.append(promo_df[promo_df['corr_counts_y']!=promo_df['corr_counts_y'].max()])
corr = corr[corr['corr_counts_y']!=1].groupby(['main_id'])['corr_counts_y'].agg(['count','max'])

In [ ]:
new_func7 = result.merge(corr,on='main_id',how='left')
new_func7 = new_func7.fillna(0)
new_func7['count'] = new_func7['count']/(new_func7['count_reviews']-1)
new_func7

In [ ]:
sort_negg = sort_negg.rename(columns={
        'month_on_site':'months_on_site',
        'count_reviews':'reviews_number',
        'main_review':'review',
        'main_rate':'rating',
        'date_median':'minutes_between_reviews',
        'rates_neg_share':'share_of_negative_ratings',
        'q70_ratio':'similarity_of_client_reviews',
        'med_other_review_len':'median_length_of_other_reviews',
        'med_count_photo':'median_number_of_photos',
        'max':'max_similarity_of_products',
        'count':'share_similarity_of_products',
        'main_simillar':'similarity_of_main_reviews',
        'std':'std_similarity_of_client_reviews'})
col_to_check = ['months_on_site','headphones_id','reviews_number','minutes_between_reviews',
                                  'date_std','share_of_negative_ratings',
                                  'similarity_of_client_reviews','median_length_of_other_reviews',
                                  'median_number_of_photos','var_other_review_len',
                                  'var_other_rate','share_similarity_of_products',
                                  'max_similarity_of_products']
final = sort_negg[col_to_check].drop_duplicates()
final_df = sort_negg.loc[final.index]

# подготовка тестового датафрейма для gpt3

In [ ]:
gpt3_df = final_df.copy()
for i in ['share_of_negative_ratings',
          'similarity_of_client_reviews',
          'std_similarity_of_client_reviews',
          'var_other_rate',
          'share_similarity_of_products']:
    gpt3_df[i] = gpt3_df[i].round(2)
    
for i in ['headphones_id',
          'sorted',
          'months_on_site',
          'reviews_number',
          'main_review_len',
          'rating',
          'minutes_between_reviews',
          'date_std',
          'median_length_of_other_reviews',
          'median_number_of_photos',
          'var_other_review_len',
          'max_similarity_of_products']:
    gpt3_df[i] = gpt3_df[i].astype(int)
gpt3_df = gpt3_df.sort_values(by=['share_similarity_of_products',
                                  'similarity_of_client_reviews',
                                  'similarity_of_main_reviews',
                                  'minutes_between_reviews'], ascending=[False, False,False,False])

In [ ]:
checc = gpt3_df.copy()

In [ ]:
#checc.to_excel('final_df_to_save.xlsx')

In [ ]:
def prompt(index_start,index_stop,fake,true):
    other = checc.sorted.to_list()
    for item in other:
        if item in fake:
            other.remove(item)
    
    prompt_message = f'Given client data: {checc.columns.to_list()[1:]}. Determine if a review is fake or not, based on the review and customer data. Label the data.\n' 
    
    prompt_message += '''Example: \n'''  
    prompt_message += 'Inputs: \n'
    
    for i in fake: 
        prompt_message += f'''data {i}: {list(checc[checc.sorted==i][list(checc.columns)[1:]].values[0])} \n'''
    for i in true:
        prompt_message += f'''data {i}: {list(checc[checc.sorted==i][list(checc.columns)[1:]].values[0])} \n'''
    
    prompt_message += '\nOutputs: \n'
    
    for i in fake:
        prompt_message += f'label {i}: Fake \n'
    for i in true:
        prompt_message += f'label {i}: True \n'
        
    prompt_message += '\nInputs: \n'
    
    for i in other[index_start:index_stop]:
        prompt_message += f'''data {i}: {list(checc[checc.sorted==i][list(checc.columns)[1:]].values[0])} \n'''
        
    prompt_message += '\nOutputs: \n'
    
    return prompt_message

In [ ]:
fake = [949,3819,2044]
true = [3506,4230,3533]

In [ ]:
import openai
openai.api_key = 'hiden_due_privacy'
model_engine = "text-davinci-003" 
temp = 0 

In [ ]:
for j in tqdm(list(range(3468, 3600))[::4]):
    i = j - 4
    if i < 0:
        #break
    print(i, j)
    try:
        response = openai.Completion.create(
            engine=model_engine,
            prompt=prompt(i,j,fake,true),
            temperature=temp,
            max_tokens=50)

        resp = response.choices[0].text.split('\n')
        data = [tuple(x.strip().split(": ")) for x in resp]
        promo_df = pd.DataFrame(data,columns=['label', 'value'])
        promo_df['label'] = promo_df['label'].str.replace('label ', '').astype(int)
        labeled_data = labeled_data.append(promo_df)
    except:
        try:
            print('WARNING_lvl1')
            response = openai.Completion.create(
                engine=model_engine,
                prompt=prompt(i,j,fake[:-1],true[:-1]),
                temperature=temp,
                max_tokens=50)
    
            resp = response.choices[0].text.split('\n')
            data = [tuple(x.strip().split(": ")) for x in resp]
            promo_df = pd.DataFrame(data,columns=['label', 'value'])
            promo_df['label'] = promo_df['label'].str.replace('label ', '').astype(int)
            labeled_data = labeled_data.append(promo_df)
        except:
            print('WARNING_lvl2')
            response = openai.Completion.create(
                engine=model_engine,
                prompt=prompt(i,j,fake[:-2],true[:-2]),
                temperature=temp,
                max_tokens=50)

            resp = response.choices[0].text.split('\n')
            data = [tuple(x.strip().split(": ")) for x in resp]
            promo_df = pd.DataFrame(data,columns=['label', 'value'])
            promo_df['label'] = promo_df['label'].str.replace('label ', '').astype(int)
            labeled_data = labeled_data.append(promo_df)

In [ ]:
#labeled_data.to_excel('label_final.xlsx')

In [ ]:
labeled_data['value'].value_counts()/labeled_data['value'].value_counts().sum()

In [ ]:
labeled_data = labeled_data.drop_duplicates(subset='label')

In [ ]:
gpt3_df[gpt3_df['sorted'].isin(labeled_data[labeled_data['value']=='Fake']['label'])].describe()

In [ ]:
gpt3_df[gpt3_df['sorted'].isin(labeled_data[labeled_data['value']=='True']['label'])].describe()

In [ ]:
labeled_try_df = gpt3_df.copy()
labeled_try_df = labeled_try_df.merge(labeled_data,on='main_id',how='left')
drop = labeled_try_df[['months_on_site','reviews_number',
              'review','minutes_between_reviews',
              'similarity_of_main_reviews','max_similarity_of_products']].duplicated()
labeled_try_df = labeled_try_df[~drop]
labeled_try_df.to_excel('detection_df.xlsx')